In [ ]:
import pandas as pd
import math
import numpy as np

pd.set_option('display.max_columns', None) 
mWindow = 10 #in sec, but should eventually be in ms
fWindow = 2 #in sec, but should eventually be in ms

In [ ]:
rawData = pd.read_csv('./raw-data.csv')
fullData = pd.DataFrame(columns=['timestamp','m1','m2','m3','m4'])

In [ ]:
rawData.head(n=11)

In [ ]:
#calculate m
i=0
for row in rawData.itertuples():
    m1 = math.sqrt(row[2]**2 + row[3]**2 + row[4]**2)
    m2 = math.sqrt(row[5]**2 + row[6]**2 + row[7]**2)
    m3 = math.sqrt(row[8]**2 + row[9]**2 + row[10]**2)
    m4 = math.sqrt(row[11]**2 + row[12]**2 + row[13]**2) 
    rawData.at[i, 'm1'] = m1
    rawData.at[i, 'm2'] = m2
    rawData.at[i, 'm3'] = m3
    rawData.at[i, 'm4'] = m4
    i += 1

In [ ]:
#subtract the mean from the data within a time window centered around the current sample
for row in rawData.itertuples():
    centerTime = row.timestamp
    i = row.Index
    laterNotDone = True
    beforeNotDone = True
    alxList = [row['arm_left_x']]
    alyList = [row['arm_left_y']]
    alzList = [row['arm_left_z']]
    arxList = [row['arm_right_x']]
    aryList = [row['arm_right_y']]
    arzList = [row['arm_right_z']]
    llxList = [row['leg_left_x']]
    llyList = [row['leg_left_y']]
    llzList = [row['leg_left_z']]
    lrxList = [row['leg_right_x']]
    lryList = [row['leg_right_y']]
    lrzList = [row['leg_right_z']]
    listNames = ['alxList','alyList','alzList','arxList','aryList','arzList','llxList','llyList',
                   'llzList','lrxList','lryList','lrzList'] 
    columnList = [alxList,alyList,alzList,arxList,aryList,arzList,llxList,llyList,
                  llzList,lrxList,lryList,lrzList] 
    x = 1
    
    while beforeNotDone and i-x >= 0:
        if centerTime - rawData.loc[i-x].timestamp <= mWindow/2 :
            nameIndex = 0
            for clist in columnList:
                column = listNames[nameIndex]
                clist.append(rawData.loc[i-x][column])
                nameIndex += 1
            x += 1
        else:
            beforeNotDone = False
            x = 1
            
    while laterNotDone and i + x < len(rawData):
        if rawData.loc[i+x].timestamp - centerTime <= mWindow/2 :
            nameIndex = 0
            for clist in columnList:
                column = listNames[nameIndex]
                clist.append(rawData.loc[i-x][column])
                nameIndex += 1
            x += 1
        else:
            laterNotDone = False
            x = 1
    
    alxc = row.arm_left_x - np.mean(alxList)
    alyc = row.arm_left_y - np.mean(alyList)
    alzc = row.arm_left_z - np.mean(alzList)
    arxc = row.arm_right_x - np.mean(arxList)
    aryc = row.arm_right_y - np.mean(aryList)
    arzc = row.arm_right_z - np.mean(arzList)
    llxc = row.leg_left_x - np.mean(llxList)
    llyc = row.leg_left_y - np.mean(llyList)
    llzc = row.leg_left_z - np.mean(llzList)
    lrxc = row.leg_right_x - np.mean(lrxList)
    lryc = row.leg_right_y - np.mean(lryList)
    lrzc = row.leg_right_z - np.mean(lrzList)
    
            
    mla = math.sqrt(alxc**2 + alyc**2 + alzc**2)
    mra = math.sqrt(arxc**2 + aryc**2 + arzc**2)
    mll = math.sqrt(llxc**2 + llyc**2 + llzc**2)
    mrl = math.sqrt(lrxc**2 + lryc**2 + lrzc**2)
    
    fullData = fullData.append({'raw_id':row.id, 'timestamp':row.Timestamp, 
                                'left_arm_x_calibrated':alxc,'left_arm_y_calibrated':alyc,'left_arm_z_calibrated':alzc,
                                'right_arm_x_calibrated':arxc,'right_arm_y_calibrated':aryc,'right_arm_z_calibrated':arzc,
                                'left_leg_x_calibrated':llxc,'left_leg_y_calibrated':llyc,'left_leg_z_calibrated':llzc,
                                'right_lef_x_calibrated':lrxc,'right_leg_y_calibrated':lryc,'right_leg_z_calibrated':lrzc,
                                'm_left_arm': mla, 'm_right_arm':mra, 'm_left_leg':mll, 'm_right_leg':mrl}, 
                                 ignore_index=True)

In [ ]:
#calculate max and prod for each row
for row in fullData.itertuples():  
    i = row.Index
    max_mla_mra = max(row.m_left_arm, row.m_right_arm)
    max_mll_mrl = max(row.m_left_leg, row.m_right_leg)
    max_mla_mll = max(row.m_left_arm, row.m_left_leg)
    max_mra_mrl = max(row.m_right_arm, row.m_right_leg)
    max_mra_mla_mrl_mll = max(row.m_right_arm, row.m_left_arm, row.m_right_leg, row.m_left_leg)
    prod_mla_mra = row.m_left_arm * row.m_right_arm
    prod_mll_mrl = row.m_left_leg * row.m_right_leg
    prod_mla_mll = row.m_left_arm * row.m_left_leg
    prod_mra_mrl = row.m_right_arm * row.m_right_leg
    prod_mra_mla_mrl_mll = row.m_right_arm * row.m_left_arm * row.m_right_leg * row.m_left_leg
    

    fullData.at[i, 'max_mla_mra'] = max_mla_mra
    fullData.at[i, 'max_mll_mrl'] = max_mll_mrl
    fullData.at[i, 'max_mla_mll'] = max_mla_mll
    fullData.at[i, 'max_mra_mrl'] = max_mra_mrl
    fullData.at[i, 'max_mra_mla_mrl_mll'] = max_mra_mla_mrl_mll
    fullData.at[i, 'prod_mla_mra'] = prod_mla_mra
    fullData.at[i, 'prod_mll_mrl'] = prod_mll_mrl
    fullData.at[i, 'prod_mla_mll'] = prod_mla_mll
    fullData.at[i, 'prod_mra_mrl'] = prod_mra_mrl
    fullData.at[i, 'prod_mra_mla_mrl_mll'] = prod_mra_mla_mrl_mll

In [ ]:
#calculate the mean, max, min, standard deviation, and z-score 
#from data within a time window centered around the current sample
for row in fullData.itertuples(): 
    i = row.Index
    centerTime = row.timestamp
    laterNotDone = True
    beforeNotDone = True
    x = 1
    
    m_left_arm = [row.m_left_arm]
    m_right_arm = [row.m_right_arm]
    m_left_leg = [row.m_left_leg]
    m_right_leg = [row.m_right_leg]
    
    max_mla_mra = [row.max_mla_mra]
    max_mll_mrl = [row.max_mll_mrl]
    max_mla_mll = [row.max_mla_mll]
    max_mra_mrl = [row.max_mra_mrl]
    max_mra_mla_mrl_mll = [row.max_mra_mla_mrl_mll]
    
    prod_mla_mra = [row.prod_mla_mra]
    prod_mll_mrl = [row.prod_mll_mrl]
    prod_mla_mll = [row.prod_mla_mll]
    prod_mra_mrl = [row.prod_mra_mrl]
    prod_mra_mla_mrl_mll = [row.prod_mra_mla_mrl_mll]
    
    listNames = ['m_left_arm','m_right_arm','m_left_leg','m_right_leg',
                 'max_mla_mra','max_mll_mrl','max_mla_mll','max_mra_mrl','max_mra_mla_mrl_mll',
                 'prod_mla_mra','prod_mll_mrl','prod_mla_mll','prod_mra_mrl','prod_mra_mla_mrl_mll'] 
    featureList = [m_left_arm,m_right_arm,m_left_leg,m_right_leg,
                   max_mla_mra,max_mll_mrl,max_mla_mll,max_mra_mrl,max_mra_mla_mrl_mll,
                   prod_mla_mra,prod_mll_mrl,prod_mla_mll,prod_mra_mrl,prod_mra_mla_mrl_mll] 
    
    while beforeNotDone and i - x  >= 0:
        if centerTime - fullData.loc[i-x].timestamp <= fWindow/2 :
            nameIndex = 0
            for ftList in featureList:
                ft = listNames[nameIndex]
                ftList.append(fullData.loc[i-x][ft])
                nameIndex += 1
            x += 1
        else:
            beforeNotDone = False
            x = 1
            
    while laterNotDone and i + x < len(fullData):
        if fullData.loc[i+x].timestamp - centerTime <= fWindow/2 :
            nameIndex = 0
            for ftList in featureList:
                ft = listNames[nameIndex]
                ftList.append(fullData.loc[i-x][ft])
                nameIndex += 1
            x += 1
        else:
            laterNotDone = False
            x = 1
            
    nameIndex = 0        
    for ftList in featureList:
        ft = listNames[nameIndex]
        std = np.std(ftList)
        mean = np.mean(ftList)
        meanft = 'time_mean_of_' + ft
        maxft = 'time_max_of_' + ft
        minft = 'time_min_of_' + ft
        sdft = 'time_sd_of_' + ft
        zft = 'time_z_of_' + ft
        fullData.at[i, meanft] = mean
        fullData.at[i, maxft] = max(ftList)
        fullData.at[i, minft] = min(ftList)
        fullData.at[i, sdft] = std
        fullData.at[i, zft] = np.divide(fullData.loc[i][ft] - mean, std, where=std!=0)
        nameIndex += 1
                
fullData = fullData.drop('timestamp', axis = 1)

In [ ]:
fullData.head()

In [ ]:
fullData.to_csv('test_output.csv', index=False)